In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Web Scraping for the Data from Wiki
#### Define URL, and then use get method of the api, get contents using .content

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802'
data = requests.get( url ).content

#### Create Soup Object

In [3]:
soup = BeautifulSoup(data, 'html.parser')

#### Create DataFrame From Table, if Borough is not assigned drop that row, else if neighborhood is not assigned make it same as borough

#### Use tr tag to get table rows, td to get table row data, .text to get text of the element, .strip to get rid of spaces


In [4]:
table_contents = []
table = soup.find('table')
for row in table.findAll('tr')[1:]:
    row_elems = row.findAll('td')
    cell = {}
    pc = row_elems[0].text.strip()
    br = row_elems[1].text.strip()
    nh = row_elems[2].text.strip()
    if( br == 'Not assigned' ):        
        pass
    elif( nh =='Not assigned' ):
        nh = br
    else:
        cell['PostalCode'] = pc[:3]
        cell['Borough'] = br
        cell['Neighborhood'] = nh
        table_contents.append(cell)
#Create data Frame
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Print the Shape, there are 103 rows and 3 columns

In [6]:
df.shape

(103, 3)

# Attach GeoSpatial Data

##### Install Geocoder

In [11]:
#!pip install geocoder
import geocoder # import geocoder

#### Define a function to get latitude, longitude and I will apply this to the dataframe  to each row's postal code

In [14]:

def get_coordinates( pc ):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
        print(g)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return([latitude,longitude])

#### The Api doesnt seem to work for me

In [16]:
#df['Location'] = df['PostalCode'].apply(get_coordinates)

#### Use the Stored Data from CSV file of locations

In [20]:
loc_data = pd.read_csv('Geospatial_Coordinates.csv')

#### Join the two data frames on the PostalCode Key 

In [31]:
df_loc = df.set_index('PostalCode').join(loc_data.set_index('PostalCode'), how = 'inner' ).reset_index()

In [33]:
df_loc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
